### Data cleaning

In [49]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import bz2

In [50]:
INIT_INPUT_DIR = '../metaml/data/'
DATA_ORIGINAL_DIR = '../data-original/'
DATA_DIR = '../data/'
LOG_DIR = '../logs/'

ABUNDANCE_FILE = 'abundance.txt'

In [51]:
def exists(folder, file):
    """
    Check if a file exists in a folder
    """
    return os.path.exists(os.path.join(folder, file))

In [52]:
def convert_bz2_txt(file_name):
    """
    Opens bz2 file, reads the zipped data and writes it into the data directory
    Returns - the path to the unzipped file
    """
    input_file_path = os.path.join(INIT_INPUT_DIR, file_name)

    with bz2.BZ2File(input_file_path, 'rb') as f:
        data = f.read()
    
    output_file_path = os.path.join(DATA_ORIGINAL_DIR, file_name)

    with open(output_file_path, 'wb') as f:
        f.write(data)

    return output_file_path

In [53]:
abundance_file_path_og = None

if(exists(DATA_ORIGINAL_DIR, ABUNDANCE_FILE)):
    abundance_file_path_og = os.path.join(DATA_ORIGINAL_DIR, ABUNDANCE_FILE)
    print(f"File {abundance_file_path_og} already exists, skipping conversion...")
else:
    print("Converting file...")
    abundance_file_path_og = convert_bz2_txt(ABUNDANCE_FILE + '.bz2')
    print("Stored in ", abundance_file_path_og)

assert abundance_file_path_og is not None

File ../data-original/abundance.txt already exists, skipping conversion...


In [54]:
def read_txt(folder_name, file_name):
    """
    Reads the data from the txt file
    Returns - the data as a pandas dataframe
    """
    input_file_path = os.path.join(folder_name, file_name)
    data = pd.read_csv(input_file_path, sep='\t', index_col=0, header=None)
    return data

def store_csv(data: pd.DataFrame, folder_name, file_name):
    """
    Stores the data into a csv file in the data directory
    Returns - the path to the csv file
    """
    output_file_path = os.path.join(folder_name, file_name)
    data.to_csv(output_file_path, index=False)
    return output_file_path

In [55]:
abundance_file_path = None

if(exists(DATA_DIR, ABUNDANCE_FILE)):
    abundance_file_path = os.path.join(DATA_DIR + ABUNDANCE_FILE)
    print(f"File {abundance_file_path} already exists, skipping conversion...")
else:
    print("Converting file...")
    data = read_txt(DATA_ORIGINAL_DIR, ABUNDANCE_FILE)
    data.describe()
    data_transposed = data.transpose()
    abundance_file_path = store_csv(data_transposed, DATA_DIR, ABUNDANCE_FILE)
    print("Stored in ", abundance_file_path)

File ../data/abundance.txt already exists, skipping conversion...


In [56]:
data = pd.read_csv(abundance_file_path)

if(not exists(LOG_DIR, 'abundance_columns.txt')):
    with open(os.path.join(LOG_DIR, 'abundance_columns.txt'), 'w') as f:
        for col in data.columns:
            f.write(col + '\n')

/tmp/ipykernel_657589/2935189533.py:1: DtypeWarning: Columns (2,5,9,20,26,42,47,51,52,59,61,68,76,77,78,79) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(abundance_file_path)


In [57]:
# First 211 columns are metadata
metadata = data.iloc[:, :211]

data_only_abundance_values = data.iloc[:, 211:]

In [58]:
if(not exists(DATA_DIR, 'abundance_values.csv')):
    print("Storing abundance values...")
    data_only_abundance_values.to_csv(os.path.join(DATA_DIR, 'abundance_values.csv'), index=False)

if(not exists(DATA_DIR, 'metadata.csv')):
    print("Storing metadata...")
    metadata.to_csv(os.path.join(DATA_DIR, 'metadata.csv'), index=False)

if(not exists(LOG_DIR, "metadata_columns.txt")):
    print("Storing metadata columns...")
    with open(os.path.join(LOG_DIR, "metadata_columns.txt"), 'w') as f:
        for col in metadata.columns:
            f.write(col + ", " + metadata[col].dtype.name + '\n')

if(not exists(LOG_DIR, 'abundance_values_columns.txt')):
    print("Storing abundance values columns...")
    with open(os.path.join(LOG_DIR, 'abundance_values_columns.txt'), 'w') as f:
        for col in data_only_abundance_values.columns:
            f.write(col + ", " + data_only_abundance_values[col].dtype.name + '\n')

In [59]:
# check for NAN values

nan_values = data_only_abundance_values.isna().sum().sum()
print(f"Number of NAN values: {nan_values}")

Number of NAN values: 0


In [60]:
metadata.head()

,dataset_name,sampleID,subjectID,bodysite,disease,age,gender,country,sequencing_technology,pubmedid,...,statins,insulin,oral_anti-diabetic_medication,years_in_sweden,tnm_stage,ajcc_stage,localization,fobt,wif-1_gene_methylation_test,group
0,Candela_Africa,H10,h10,stool,n,40,female,tanzania,Illumina,25981789,...,nd,nd,nd,nd,nd,nd,nd,nd,nd,nd
1,Candela_Africa,H11,h11,stool,n,29,female,tanzania,Illumina,25981789,...,nd,nd,nd,nd,nd,nd,nd,nd,nd,nd
2,Candela_Africa,H12,h12,stool,n,8,female,tanzania,Illumina,25981789,...,nd,nd,nd,nd,nd,nd,nd,nd,nd,nd
3,Candela_Africa,H13,h13,stool,n,34,male,tanzania,Illumina,25981789,...,nd,nd,nd,nd,nd,nd,nd,nd,nd,nd
4,Candela_Africa,H14,h14,stool,n,30,male,tanzania,Illumina,25981789,...,nd,nd,nd,nd,nd,nd,nd,nd,nd,nd


In [61]:
# Store the describe content in a file

if(not exists(LOG_DIR, 'metadata_describe.txt')):
    with open(os.path.join(LOG_DIR, 'metadata_describe.txt'), 'w') as f:
        # write properties of each column of metadata in a file
        for i, col in enumerate(metadata.columns):
            f.write(f"Number: {i}, Column: {col}\n")
            f.write(str(metadata[col].describe()) + '\n\n')

In [62]:
# is there any unique column in metadata?

unique_columns = []
for col in metadata.columns:
    if len(metadata[col].unique()) == metadata.shape[0]:
        unique_columns.append(col)

print(f"Unique columns: {unique_columns}")

Unique columns: []


In [63]:
metadata['dataset_name'].unique()

array(['Candela_Africa', 'Chatelier_gut_obesity', 'doyle_bt2', 'hmp',
       'hmpii', 'Loman2013_EcoliOutbreak_DNA_HiSeq',
       'Loman2013_EcoliOutbreak_DNA_MiSeq', 'metahit',
       'Neilsen_genome_assembly', 'Psoriasis_2014',
       'Quin_gut_liver_cirrhosis', 'Segre_Human_Skin',
       'Tito_subsistence_gut', 't2dmeta_long', 't2dmeta_short',
       'VerticalTransmissionPilot', 'WT2D',
       'Zeller_fecal_colorectal_cancer'], dtype=object)

In [64]:
metadata['disease'].unique()

array(['n', 'obesity', 'leaness', 'nd', 'stec2-positive', ' -',
       'ibd_ulcerative_colitis', 'ibd_crohn_disease', 'n_relative', 'y',
       'cirrhosis', 'obese', 'overweight', '-', 'underweight', 't2d',
       'impaired_glucose_tolerance', 'cancer', 'small_adenoma',
       'large_adenoma'], dtype=object)

In [65]:
# add a new column in a new dataframe called UniqueID which is a combination of dataset_name, sampleID and subjectID

metadata_new = metadata.copy()

# Convert all columns to string
metadata_new['dataset_name'] = metadata_new['dataset_name'].astype(str)
metadata_new['sampleID'] = metadata_new['sampleID'].astype(str)
metadata_new['subjectID'] = metadata_new['subjectID'].astype(str)

metadata_new['UniqueID'] = metadata_new['dataset_name'] + "_" +  metadata_new['sampleID'] + "_" + metadata_new['subjectID']

# show UniqueID column in the beginning
metadata_new = metadata_new[['UniqueID'] + [col for col in metadata_new.columns if col != 'UniqueID']]
metadata_new.describe(include='all')

# Store new metadata in a csv file
if(not exists(DATA_DIR, 'metadata_new.csv')):
    metadata_new.to_csv(os.path.join(DATA_DIR, 'metadata_new.csv'), index=False)

In [66]:
data_only_abundance_values.head()

,k__Archaea,k__Archaea|p__Euryarchaeota,k__Archaea|p__Euryarchaeota|c__Methanobacteria,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii|t__Methanobrevibacter_smithii_unclassified,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_unclassified,k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanosphaera,...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Enterococcaceae|g__Enterococcus|s__Enterococcus_gilvus|t__Enterococcus_gilvus_unclassified,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus|s__Lactobacillus_otakiensis,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus|s__Lactobacillus_otakiensis|t__GCF_000415925,k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Peptococcaceae,k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Peptococcaceae|g__Desulfotomaculum,k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Peptococcaceae|g__Desulfotomaculum|s__Desulfotomaculum_ruminis,k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Peptococcaceae|g__Desulfotomaculum|s__Desulfotomaculum_ruminis|t__GCF_000215085,k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Faecalibacterium|s__Faecalibacterium_prausnitzii|t__GCF_000209855,k__Bacteria|p__Firmicutes|c__Negativicutes|o__Selenomonadales|f__Veillonellaceae|g__Megasphaera|s__Megasphaera_sp_BV3C16_1,k__Bacteria|p__Firmicutes|c__Negativicutes|o__Selenomonadales|f__Veillonellaceae|g__Megasphaera|s__Megasphaera_sp_BV3C16_1|t__GCF_000478965
0,0.24169,0.24169,0.24169,0.24169,0.24169,0.24169,0.24169,0.24169,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.50621,0.50621,0.50621,0.50621,0.50621,0.50621,0.50621,0.50621,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.30522,0.30522,0.30522,0.30522,0.30522,0.30522,0.30522,0.30522,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.40133,0.40133,0.40133,0.40133,0.40133,0.35994,0.35994,0.35994,0.00000,0.04139,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.17479,0.17479,0.17479,0.17479,0.17479,0.16331,0.15280,0.15280,0.01051,0.01148,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
data_only_abundance_values_new = data_only_abundance_values.copy()
data_only_abundance_values_new['UniqueID'] = metadata_new['UniqueID']

# Show UniqueID column in the beginning
data_only_abundance_values_new = data_only_abundance_values_new[['UniqueID'] + [col for col in data_only_abundance_values_new.columns if col != 'UniqueID']]
data_only_abundance_values_new.head()

# Store new data in a csv file
if(not exists(DATA_DIR, 'abundance_values_new.csv')):
    data_only_abundance_values_new.to_csv(os.path.join(DATA_DIR, 'abundance_values_new.csv'), index=False)

In [68]:
# Clean every data in memory except metadata_new and data_only_abundance_values_new
del data
del metadata
del data_only_abundance_values

In [70]:
metadata_new.head()

,UniqueID,dataset_name,sampleID,subjectID,bodysite,disease,age,gender,country,sequencing_technology,...,statins,insulin,oral_anti-diabetic_medication,years_in_sweden,tnm_stage,ajcc_stage,localization,fobt,wif-1_gene_methylation_test,group
0,Candela_Africa_H10_h10,Candela_Africa,H10,h10,stool,n,40,female,tanzania,Illumina,...,nd,nd,nd,nd,nd,nd,nd,nd,nd,nd
1,Candela_Africa_H11_h11,Candela_Africa,H11,h11,stool,n,29,female,tanzania,Illumina,...,nd,nd,nd,nd,nd,nd,nd,nd,nd,nd
2,Candela_Africa_H12_h12,Candela_Africa,H12,h12,stool,n,8,female,tanzania,Illumina,...,nd,nd,nd,nd,nd,nd,nd,nd,nd,nd
3,Candela_Africa_H13_h13,Candela_Africa,H13,h13,stool,n,34,male,tanzania,Illumina,...,nd,nd,nd,nd,nd,nd,nd,nd,nd,nd
4,Candela_Africa_H14_h14,Candela_Africa,H14,h14,stool,n,30,male,tanzania,Illumina,...,nd,nd,nd,nd,nd,nd,nd,nd,nd,nd


In [71]:
metadata_new['bodysite'].unique()

array(['stool', 'nd', 'tongue_dorsum', 'buccal_mucosa',
       'supragingival_plaque', 'anterior_nares', 'posterior_fornix',
       'l_retroauricular_crease', 'r_retroauricular_crease', 'saliva',
       'keratinized_gingiva', 'palatine_tonsils', 'throat',
       'subgingival_plaque', 'vaginal_introitus', 'mid_vagina',
       'hard_palate', 'gomito_dx', 'gomito_sx',
       'solco_retroauricolare_dx', 'solco_retroauricolare_sx', 'skin',
       'mother_faeces', 'infant_faeces', 'milk'], dtype=object)

In [72]:
for col in metadata_new.columns:
    print(f"Column: {col}, Type: {metadata_new[col].dtype.name}")

Column: UniqueID, Type: object
Column: dataset_name, Type: object
Column: sampleID, Type: object
Column: subjectID, Type: object
Column: bodysite, Type: object
Column: disease, Type: object
Column: age, Type: object
Column: gender, Type: object
Column: country, Type: object
Column: sequencing_technology, Type: object
Column: pubmedid, Type: object
Column: camp, Type: object
Column: paired_end_insert_size, Type: object
Column: read_length, Type: object
Column: total_reads, Type: object
Column: matched_reads, Type: object
Column: uniquely_matching_reads, Type: object
Column: uniquely_matched_reads, Type: object
Column: gene_number, Type: object
Column: gene_number_for_11m_uniquely_matched_reads, Type: object
Column: hitchip_probe_number, Type: object
Column: bmi, Type: object
Column: gene_count_class, Type: object
Column: hitchip_probe_class, Type: object
Column: #SampleID, Type: object
Column: rna_sampleid, Type: object
Column: infant_gender, Type: object
Column: infant_gestation_weeks,